# Awari - Aula 7 - Kafka

1. Importar as diferenças que são criadas nos dados da pasta /exercicios/municipios-estados/streaming/. A cada vez que um novo arquivo for adicionado, o mesmo deve ser importado para a pasta da UF correspondente e adicionado ao fim do arquivo cidades.csv.
2. Utilizar Apache Kafka para fazer o mesmo processo, mas de maneira automatizada.

## Tratamentos iniciais

In [1]:
import pandas as pd

In [2]:
df_estados = pd.read_csv("../arquivos/estados.csv")

In [3]:
df_estados.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


In [4]:
df_municipios = pd.read_csv("../arquivos/municipios.csv")

In [5]:
df_municipios.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [6]:
df_municipios.merge(df_estados[['uf', 'codigo_uf']], on="codigo_uf")

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
2,5200134,Acreúna,-17.3960,-50.3749,0,52,9645,64,America/Sao_Paulo,GO
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO
...,...,...,...,...,...,...,...,...,...,...
5565,3306107,Valença,-22.2445,-43.7129,0,33,5921,24,America/Sao_Paulo,RJ
5566,3306156,Varre-Sai,-20.9276,-41.8701,0,33,2917,22,America/Sao_Paulo,RJ
5567,3306206,Vassouras,-22.4059,-43.6686,0,33,5923,24,America/Sao_Paulo,RJ
5568,3306305,Volta Redonda,-22.5202,-44.0996,0,33,5925,24,America/Sao_Paulo,RJ


In [7]:
df = df_municipios.merge(df_estados[['uf', 'codigo_uf']], on="codigo_uf")

## MinIO

In [8]:
#!pip install boto3

In [9]:
import boto3

In [10]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='hxhFDOAzIu8nZM33',
    aws_secret_access_key='TKHENO4SuG4mLURfD0sxKXGvPokV9GT1',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [11]:
for uf in df.uf.unique():
    aux = df[df.uf == uf]
    client.put_object(Bucket='aula-06', Key=f'{uf}/municipios.csv', Body=aux.to_csv(index=False))

## Sem Kafka

In [12]:
estado_novo = pd.read_csv("../arquivos/estados_streaming.csv")

In [13]:
estado_novo

,codigo_uf,uf,nome,latitude,longitude,regiao
0,101,AWARI-1,Awari Estado 1,-10.83,-63.34,Norte
1,102,AWARI-2,Awari Estado 2,-8.77,-70.55,Norte


In [14]:
municipio_novo = pd.read_csv("../arquivos/municipios_streaming.csv")	

In [15]:
municipio_novo

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,9900001,Minha cidade 1,-16.7573,-49.4412,0,52,1050,100,America/Sao_Paulo
1,9900002,Minha cidade 2,-18.4831,-47.3916,0,31,4001,101,America/Sao_Paulo


In [16]:
estado_novo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo_uf  2 non-null      int64  
 1   uf         2 non-null      object 
 2   nome       2 non-null      object 
 3   latitude   2 non-null      float64
 4   longitude  2 non-null      float64
 5   regiao     2 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 224.0+ bytes


In [17]:
df_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo_uf  27 non-null     int64  
 1   uf         27 non-null     object 
 2   nome       27 non-null     object 
 3   latitude   27 non-null     float64
 4   longitude  27 non-null     float64
 5   regiao     27 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.4+ KB


In [18]:
municipio_novo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   2 non-null      int64  
 1   nome          2 non-null      object 
 2   latitude      2 non-null      float64
 3   longitude     2 non-null      float64
 4   capital       2 non-null      int64  
 5   codigo_uf     2 non-null      int64  
 6   siafi_id      2 non-null      int64  
 7   ddd           2 non-null      int64  
 8   fuso_horario  2 non-null      object 
dtypes: float64(2), int64(5), object(2)
memory usage: 272.0+ bytes


In [19]:
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   5570 non-null   int64  
 1   nome          5570 non-null   object 
 2   latitude      5570 non-null   float64
 3   longitude     5570 non-null   float64
 4   capital       5570 non-null   int64  
 5   codigo_uf     5570 non-null   int64  
 6   siafi_id      5570 non-null   int64  
 7   ddd           5570 non-null   int64  
 8   fuso_horario  5570 non-null   object 
dtypes: float64(2), int64(5), object(2)
memory usage: 391.8+ KB


In [20]:
aux1 = pd.concat([df_estados, estado_novo], axis=0)

In [21]:
aux1.codigo_uf.unique()

array([ 11,  12,  13,  14,  15,  16,  17,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  31,  32,  33,  35,  41,  42,  43,  50,  51,  52,
        53, 101, 102])

In [22]:
aux2 = pd.concat([df_municipios, municipio_novo], axis=0)

In [23]:
aux = pd.merge(aux2, aux1[['uf', 'codigo_uf']], on="codigo_uf")

In [24]:
aux.loc[aux.nome=='Minha cidade 1']

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
246,9900001,Minha cidade 1,-16.7573,-49.4412,0,52,1050,100,America/Sao_Paulo,GO


In [25]:
def concat_df(antigo, novo):
    df = pd.concat([antigo, novo], axis=0)
    df.drop_duplicates(inplace=True)
    return df

In [26]:
def merge_df(municipio, estado):
    df = municipio.merge(estado[['uf', 'codigo_uf']], on="codigo_uf")
    return df

In [27]:
def summary(municipio_antigo, municipio_novo, estado_antigo, estado_novo):
    estado = concat_df(estado_antigo, estado_novo)
    municipio = concat_df(municipio_antigo, municipio_novo)
    df = merge_df(municipio, estado)
    return df

In [28]:
df = summary(df_municipios, municipio_novo, df_estados, estado_novo)

In [29]:
df

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
2,5200134,Acreúna,-17.3960,-50.3749,0,52,9645,64,America/Sao_Paulo,GO
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO
...,...,...,...,...,...,...,...,...,...,...
5567,3306107,Valença,-22.2445,-43.7129,0,33,5921,24,America/Sao_Paulo,RJ
5568,3306156,Varre-Sai,-20.9276,-41.8701,0,33,2917,22,America/Sao_Paulo,RJ
5569,3306206,Vassouras,-22.4059,-43.6686,0,33,5923,24,America/Sao_Paulo,RJ
5570,3306305,Volta Redonda,-22.5202,-44.0996,0,33,5925,24,America/Sao_Paulo,RJ


In [30]:
def save(df):
    for uf in df.uf.unique():
        aux = df[df.uf == uf]
        client.put_object(Bucket='aula-06', Key=f'{uf}/municipios.csv', Body=aux.to_csv(index=False))

In [31]:
save(df)

## Com Kafka

In [32]:
#!wget https://downloads.apache.org/kafka/3.6.1/kafka_2.13-3.6.1.tgz
#!tar -xvzf kafka_2.13-3.6.1.tgz

In [ ]:
!../arquivos/kafka_2.13-3.6.1/bin/zookeeper-server-start.sh -daemon ../arquivos/kafka_2.13-3.6.1/config/zookeeper.properties

In [ ]:
!../arquivos/kafka_2.13-3.6.1/bin/kafka-server-start.sh -daemon ../arquivos/kafka_2.13-3.6.1/server.properties

In [ ]:
!../arquivos/kafka_2.13-3.6.1/bin/kafka-topics.sh --create --topic novos-municipios --bootstrap-server localhost:9092

#### Observação
Não foi possível automatizar o processo com o python devido à dificuldade em configurar o ambiente do Kafka, já que o container do docker não está funcionando